In [140]:
from rllab.algos.cem import CEM
from rllab.algos.ddpg import DDPG
from rllab.algos.tnpg import TNPG
from rllab.algos.trpo import TRPO
from rllab.algos.vpg import VPG


from rllab.baselines.linear_feature_baseline import LinearFeatureBaseline
from rllab.envs.gym_env import GymEnv
from rllab.envs.normalized_env import normalize
from rllab.exploration_strategies.ou_strategy import OUStrategy
from rllab.misc.instrument import run_experiment_lite
from rllab.misc.instrument import VariantGenerator, variant
from rllab.policies.categorical_mlp_policy import CategoricalMLPPolicy
from rllab.policies.gaussian_mlp_policy import GaussianMLPPolicy
from rllab.policies.deterministic_mlp_policy import DeterministicMLPPolicy
from rllab.q_functions.continuous_mlp_q_function import ContinuousMLPQFunction
from rllab.sampler.utils import collect_data
from rllab.sampler.utils import rollout

import gym
import numpy as np
import GPy
import GPyOpt

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
import pandas as pd


import plotly
plotly.offline.init_notebook_mode() 
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

import numpy as np
import pandas as pd
import copy
import pickle
from datetime import datetime
import os


In [264]:
path = './SIM/'
test_files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'SIM' in i:
        test_files.append(path+i)
BO_test_files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'Bayesian' in i:
        BO_test_files.append(path+i)
REAL_test_files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'REAL' in i:
        REAL_test_files.append(path+i)

In [265]:
test_data = []
for file in test_files:
    with open(file, "rb") as input_file:
        e = pickle.load(input_file)
#         print(e,'\n')
        test_data.append(e)
BO_test_data = []
for file in BO_test_files:
    with open(file, "rb") as input_file:
        e = pickle.load(input_file)
        BO_test_data.append(e)
REAL_test_data = []
for file in REAL_test_files:
    with open(file, "rb") as input_file:
        e = pickle.load(input_file)
        REAL_test_data.append(e)

In [266]:
root_path = '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-400/'
paths = []
for i in os.listdir(root_path):
#     print(i)
    if 'SIM' in i:
        paths.append(i)
count = 0
for path in paths:

    full_path = root_path+path+'/progress.csv'
    df = pd.read_csv(full_path)
    if df['AverageReturn'].shape[0] < 400:
        continue
    temp = np.array(df['AverageReturn'],ndmin=2)
    if count == 0:
        train_data = temp.T
        count = count + 1
        continue
    print(train_data.shape, temp.shape)
    train_data = np.append(train_data, temp.T,axis =1)
#     print(df['AverageReturn'].shape)

(400, 1) (1, 400)
(400, 2) (1, 400)
(400, 3) (1, 400)
(400, 4) (1, 400)
(400, 5) (1, 400)
(400, 6) (1, 400)
(400, 7) (1, 400)


In [267]:
root_path = '../../data/local/BO-trpoSwimmer-v1-32-32-4000-10/back/'
paths = []
for i in os.listdir(root_path):
    if 'BO' in i:
        paths.append(i)
count = 0

for path in paths:
    full_path = root_path+path+'/progress.csv'
    df = pd.read_csv(full_path)
    if df['AverageReturn'].shape[0] < 400:
        continue
    temp = np.array(df['AverageReturn'],ndmin=2)
    if count == 0:
        BO_train_data = temp.T
        count = count + 1
        continue
    print(BO_train_data.shape, temp.shape)
    BO_train_data = np.append(BO_train_data, temp.T,axis =1)

#     print(df['AverageReturn'].shape)

(400, 1) (1, 400)
(400, 2) (1, 400)
(400, 3) (1, 400)
(400, 4) (1, 400)
(400, 5) (1, 400)


In [268]:
root_path = '../../data/local/REAL-trpo-Swimmer-v1-32-32-4000-iter-400/'
paths = []
for i in os.listdir(root_path):
    if 'REAL' in i:
        paths.append(i)
count = 0

for path in paths:
    full_path = root_path+path+'/progress.csv'
    df = pd.read_csv(full_path)
    if df['AverageReturn'].shape[0] < 400:
        continue
    temp = np.array(df['AverageReturn'],ndmin=2)
    if count == 0:
        REAL_train_data = temp.T
        count = count + 1
        continue
    print(REAL_train_data.shape, temp.shape)
    REAL_train_data = np.append(REAL_train_data, temp.T,axis =1)

#     print(df['AverageReturn'].shape)

(400, 1) (1, 400)
(400, 2) (1, 400)


In [269]:
test_data_array = np.asarray(test_data)
test_mean = np.mean(test_data_array,0)
test_std = np.std(test_data_array, axis=0)
test_upper = test_mean+test_std
test_lower = test_mean-test_std
test_lower= test_lower[::-1]

BO_test_data_array = np.asarray(BO_test_data)
BO_test_mean = np.mean(BO_test_data_array,0)
BO_test_std = np.std(BO_test_data_array, axis=0)
BO_test_upper = BO_test_mean+BO_test_std
BO_test_lower = BO_test_mean-BO_test_std
BO_test_lower= BO_test_lower[::-1]

REAL_test_data_array = np.asarray(REAL_test_data)
REAL_test_mean = np.mean(REAL_test_data_array,0)
REAL_test_std = np.std(REAL_test_data_array, axis=0)
REAL_test_upper = REAL_test_mean+REAL_test_std
REAL_test_lower = REAL_test_mean-REAL_test_std
REAL_test_lower= REAL_test_lower[::-1]

In [270]:
train_mean = np.mean(train_data,1)
train_std = np.std(train_data, axis=1)

train_upper = train_mean+train_std
train_lower = train_mean-train_std
train_lower= train_lower[::-1]

BO_train_mean = np.mean(BO_train_data,1)
BO_train_std = np.std(BO_train_data, axis=1)
BO_train_upper = BO_train_mean+BO_train_std
BO_train_lower = BO_train_mean-BO_train_std
BO_train_lower= BO_train_lower[::-1]

REAL_train_mean = np.mean(REAL_train_data,1)
REAL_train_std = np.std(REAL_train_data, axis=1)
REAL_train_upper = REAL_train_mean+REAL_train_std
REAL_train_lower = REAL_train_mean-REAL_train_std
REAL_train_lower= REAL_train_lower[::-1]

In [274]:
x_train = list(range(1,401))
x_train_rev = x_train[::-1]
x_test = np.array(range(10,410,10))
x_test_rev = x_test[::-1]

trace1 = go.Scatter(
                    y=train_mean, # Data
                    line=Line(color='rgb(0,255,80)'),
                    mode='lines', name='training' # Additional options    
                   )

trace1_std = go.Scatter(
                    x=x_train+x_train_rev, 
                    y=list(train_upper)+list(train_lower), # Data
                    fill='tozerox',
                    fillcolor='rgba(0,255,80,0.2)',
                    line=Line(color='transparent'),
                    showlegend=False, name='training' # Additional options
                   )
trace２ = go.Scatter(
                    x=x_test,
                    y=test_mean, # Data
                    line=Line(color='rgb(0,176,246)'),
                    mode='lines', name='testing' # Additional options
                   )
trace2_std = go.Scatter(
                    x=list(x_test)+list(x_test_rev), 
                    y=list(test_upper)+list(test_lower), # Data
                    fill='tozerox',
                    fillcolor='rgba(0,176,246,0.2)',
                    line=Line(color='transparent'),
                    showlegend=False, name='testing' # Additional options
                   )
trace3 = go.Scatter(
                    y=BO_train_mean, # Data
                    line=Line(color='rgb(50,150,50)'),
                    mode='lines', name='training with BO' # Additional options
                   )
trace3_std = go.Scatter(
                    x=x_train+x_train_rev, 
                    y=list(BO_train_upper)+list(BO_train_lower), # Data
                    fill='tozerox',
                    fillcolor='rgba(50,150,50,0.3)',
                    line=Line(color='transparent'),
                    showlegend=False, name='training with BO' # Additional options
                   )
trace4 = go.Scatter(
                    x=x_test,
                    y=BO_test_mean, # Data
                    line=Line(color='rgb(231,107,243)'),
                    mode='lines', name='testing with BO' # Additional options
                   )
trace4_std = go.Scatter(
                    x=list(x_test)+list(x_test_rev), 
                    y=list(BO_test_upper)+list(BO_test_lower), # Data
                    fill='tozerox',
                    fillcolor='rgba(231,107,243,0.3)',
                    line=Line(color='transparent'),
                    showlegend=False, name='testing with BO' # Additional options
                   )
trace5 = go.Scatter(
                    y=REAL_train_mean, # Data
                    line=Line(color='rgb(150,50,50)'),
                    mode='lines', name='training with REAL' # Additional options
                   )
trace5_std = go.Scatter(
                    x=x_train+x_train_rev, 
                    y=list(REAL_train_upper)+list(REAL_train_lower), # Data
                    fill='tozerox',
                    fillcolor='rgba(150,50,50,0.3)',
                    line=Line(color='transparent'),
                    showlegend=False, name='training with REAL' # Additional options
                   )
trace6 = go.Scatter(
                    x=x_test,
                    y=REAL_test_mean, # Data
                    line=Line(color='rgb(131,207,243)'),
                    mode='lines', name='testing with REAL' # Additional options
                   )
trace6_std = go.Scatter(
                    x=list(x_test)+list(x_test_rev), 
                    y=list(REAL_test_upper)+list(REAL_test_lower), # Data
                    fill='tozerox',
                    fillcolor='rgba(131,207,243,0.3)',
                    line=Line(color='transparent'),
                    showlegend=False, name='testing with REAL' # Additional options
                   )
layout = go.Layout(title='Swimmer TRPO Perfomance',
                   plot_bgcolor='rgb(255, 255,255)')

# fig = go.Figure(data=[trace3, trace3_std, trace4, trace4_std], layout=layout)
fig = go.Figure(data=[trace1, trace1_std, trace2, trace2_std, trace3, trace3_std, trace4, trace4_std], layout=layout)
# fig = go.Figure(data=[trace1, trace1_std, trace2, trace2_std, trace3, trace3_std, trace4, trace4_std,
#                      trace5, trace5_std, trace6, trace6_std,], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [19]:
with open(r"returns_Walker2d-v1_iter_500_train_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_Walker2d-v1_iter_1000_train_in_sim_with_BO", "rb") as input_file:
    e_BO = pickle.load(input_file)
    
datafile =  '../../data/local/trpo-Walker2d-v1-32-32-4000-iter-500/trpo_Walker2d-v1_32_32_4000_iter_500_train_in_sim/progress.csv'
df = pd.read_csv(datafile)
datafile2 =  '../../data/local/trpo-Walker2d-v1-32-32-4000-iter-1000/trpo_Walker2d-v1_32_32_4000_iter_1000_train_in_sim_with_BO/progress.csv'
df_BO = pd.read_csv(datafile2)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing' # Additional options
                   )
trace3 = go.Scatter(
                    y=df_BO['AverageReturn'], # Data
                    mode='lines', name='training with BO' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e_BO), # Data
                    mode='lines', name='testing with BO' # Additional options
                   )
layout = go.Layout(title='Walker2D TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [20]:
with open(r"returns_HalfCheetah-v1_iter_1000_train_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_HalfCheetah-v1_iter_1000_train_in_sim_with_BO", "rb") as input_file:
    e_BO = pickle.load(input_file)
    
datafile =  '../../data/local/trpo-HalfCheetah-v1-32-32-4000-iter-1000/trpo_HalfCheetah-v1_32_32_4000_iter_1000_train_in_sim/progress.csv'
df = pd.read_csv(datafile)
datafile2 =  '../../data/local/trpo-HalfCheetah-v1-32-32-4000-iter-1000/trpo_HalfCheetah-v1_32_32_4000_iter_1000_train_in_sim_with_BO/progress.csv'
df_BO = pd.read_csv(datafile2)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing' # Additional options
                   )
trace3 = go.Scatter(
                    y=df_BO['AverageReturn'], # Data
                    mode='lines', name='training with BO' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e_BO), # Data
                    mode='lines', name='testing with BO' # Additional options
                   )
layout = go.Layout(title='HalfCheetah TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [16]:
with open(r"returns_InvertedPendulum-v1_iter_100_train_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_InvertedPendulum-v1_iter_100_train_in_real", "rb") as input_file:
    e_real = pickle.load(input_file)
with open(r"returns_InvertedPendulum-v1_iter_1000_train_in_sim_with_BO", "rb") as input_file:
    e_BO = pickle.load(input_file)

datafile =  '../../data/local/trpo-InvertedPendulum-v1-32-32-4000-iter-1000/trpo_InvertedPendulum-v1_32_32_4000_iter_1000_train_in_sim/progress.csv'
df = pd.read_csv(datafile)

datafile2 =  '../../data/local/trpo-InvertedPendulum-v1-32-32-4000-iter-1000/trpo_InvertedPendulum-v1_32_32_4000_iter_1000_train_in_real/progress.csv'
df_real = pd.read_csv(datafile2)

datafile3 =  '../../data/local/trpo-InvertedPendulum-v1-32-32-4000-40/trpo_InvertedPendulum-v1_32_32_4000_40_train_in_sim_with_BO/progress.csv'
df_sim_BO = pd.read_csv(datafile3)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training_in_sim' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing_train_in_sim' # Additional options
                   )
trace3 = go.Scatter(
                    y=df_real['AverageReturn'], # Data
                    mode='lines', name='training_in_real' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e_real), # Data
                    mode='lines', name='testing_train_in_real' # Additional options
                   )
trace5 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e_BO), # Data
                    mode='lines', name='testing_train_in_sim_with_BO' # Additional options
                   )
trace6 = go.Scatter(
                    y=df_sim_BO['AverageReturn'], # Data
                    mode='lines', name='training_in_sim_with_BO' # Additional options
                   )


layout = go.Layout(title='InvertedPendulum TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

# fig = go.Figure(data=[trace1, trace2, trace3, trace4,trace6,trace5], layout=layout)
fig = go.Figure(data=[ trace2, trace4, trace5], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [12]:
with open(r"returns_InvertedPendulum-v1_iter_100_train_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_InvertedPendulum-v1_iter_100_train_in_real", "rb") as input_file:
    e_real = pickle.load(input_file)
with open(r"returns_InvertedPendulum-v1_iter_100_train_in_sim_with_BO", "rb") as input_file:
    e_BO = pickle.load(input_file)

datafile =  '../../data/local/trpo-InvertedPendulum-v1-32-32-4000-iter-1000/trpo_InvertedPendulum-v1_32_32_4000_iter_1000_train_in_sim/progress.csv'
df = pd.read_csv(datafile)

datafile2 =  '../../data/local/trpo-InvertedPendulum-v1-32-32-4000-iter-1000/trpo_InvertedPendulum-v1_32_32_4000_iter_1000_train_in_real/progress.csv'
df_real = pd.read_csv(datafile2)

datafile3 =  '../../data/local/trpo-InvertedPendulum-v1-32-32-4000-40/trpo_InvertedPendulum-v1_32_32_4000_40_train_in_sim_with_BO/progress.csv'
df_sim_BO = pd.read_csv(datafile3)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training_in_sim' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing_train_in_sim' # Additional options
                   )
trace3 = go.Scatter(
                    y=df_real['AverageReturn'], # Data
                    mode='lines', name='training_in_real' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e_real), # Data
                    mode='lines', name='testing_train_in_real' # Additional options
                   )
trace5 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e_BO), # Data
                    mode='lines', name='testing_train_in_sim_with_BO' # Additional options
                   )
trace6 = go.Scatter(
                    y=df_sim_BO['AverageReturn'], # Data
                    mode='lines', name='training_in_sim_with_BO' # Additional options
                   )


layout = go.Layout(title='InvertedPendulum TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2, trace3, trace4,trace6,trace5], layout=layout)
# fig = go.Figure(data=[ trace2, trace4, trace5], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [34]:
len(e2)

40

In [45]:
with open(r"returns_Swimmer-v1_iter_400_train_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_trained_in_sim_with_BO", "rb") as input_file:
    e2 = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_train_in_sim_with_BO", "rb") as input_file:
    e22 = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_trained_in_real", "rb") as input_file:
    e3 = pickle.load(input_file)
    
datafile =  '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-1000/trpo_Swimmer-v1_32_32_4000_iter_400_train_in_sim/progress.csv'
df = pd.read_csv(datafile)
datafile2 =  '../../data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_trained_in_sim_with_BO/progress.csv'
df2 = pd.read_csv(datafile2)
datafile2 =  '../../data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0012/progress.csv'
df22 = pd.read_csv(datafile2)
datafile３ =  '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-400/trpo_Swimmer-v1_32_32_4000_iter_400_trained_with_real/progress.csv'
df３ = pd.read_csv(datafile３)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training_performance_trained_in_sim' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,410,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing_performance_trained_in_sim' # Additional options
                   )
trace3 = go.Scatter(
                    y=df2['AverageReturn'], # Data
                    mode='lines', name='training_performance_trained_with_BO' # Additional options
                   )
trace33 = go.Scatter(
                    y=df22['AverageReturn'], # Data
                    mode='lines', name='training_performance_trained_with_BO' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,410,10)),
                    y=np.array(e2), # Data
                    mode='lines', name='testing_performance_trained_with_BO' # Additional options
                   )
trace44 = go.Scatter(
                    x=np.array(range(10,400,10)),
                    y=np.array(e22), # Data
                    mode='lines', name='testing_performance_trained_with_BO' # Additional options
                   )
trace5 = go.Scatter(
                    y=df3['AverageReturn'], # Data
                    mode='lines', name='training_performance_trained_in_real' # Additional options
                   )
trace6 = go.Scatter(
                    x=np.array(range(10,410,10)),
                    y=np.array(e3), # Data
                    mode='lines', name='testing_performance_trained_in_real' # Additional options
                   )
layout = go.Layout(title='Swimmer TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

# fig = go.Figure(data=[trace1, trace2], layout=layout)
fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [44]:
with open(r"returns_Swimmer-v1_iter_200_2017-trained_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_train_in_sim_with_BO", "rb") as input_file:
    e2 = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_train_in_real", "rb") as input_file:
    e3 = pickle.load(input_file)
    
datafile =  '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-200/trpo_Swimmer-v1_32_32_4000_iter_200_trained_in_sim/progress.csv'
df = pd.read_csv(datafile)
datafile2 =  '../../data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0006/progress.csv'
df2 = pd.read_csv(datafile2)
datafile３ =  '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-400/trpo_Swimmer-v1_32_32_4000_iter_400_2017_09_07_14_32_45_0001/progress.csv'
df３ = pd.read_csv(datafile３)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_in_sim' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing_perform_trained_in_sim' # Additional options
                   )
trace3 = go.Scatter(
                    y=df2['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_with_BO' # Additional options
                   )
trace33 = go.Scatter(
                    y=df22['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_with_BO' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e2), # Data
                    mode='lines', name='testing_perform_trained_with_BO' # Additional options
                   )
trace44 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e22), # Data
                    mode='lines', name='testing_perform_trained_with_BO' # Additional options
                   )
trace5 = go.Scatter(
                    y=df3['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_in_real' # Additional options
                   )
trace6 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e3), # Data
                    mode='lines', name='testing_perform_trained_in_real' # Additional options
                   )
layout = go.Layout(title='Swimmer TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2], layout=layout)
# fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [22]:
with open(r"returns_Swimmer-v1_iter_400_train_in_sim", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_train_in_sim_with_BO", "rb") as input_file:
    e2 = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_400_train_in_real", "rb") as input_file:
    e3 = pickle.load(input_file)
    
datafile =  '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-1000/trpo_Swimmer-v1_32_32_4000_iter_400_train_in_sim/progress.csv'
df = pd.read_csv(datafile)
datafile2 =  '../../data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0006/progress.csv'
df2 = pd.read_csv(datafile2)
datafile３ =  '../../data/local/trpo-Swimmer-v1-32-32-4000-iter-400/trpo_Swimmer-v1_32_32_4000_iter_400_2017_09_07_14_32_45_0001/progress.csv'
df３ = pd.read_csv(datafile３)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_in_sim' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing_perform_trained_in_sim' # Additional options
                   )
trace3 = go.Scatter(
                    y=df2['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_with_BO' # Additional options
                   )
trace33 = go.Scatter(
                    y=df22['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_with_BO' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e2), # Data
                    mode='lines', name='testing_perform_trained_with_BO' # Additional options
                   )
trace44 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e22), # Data
                    mode='lines', name='testing_perform_trained_with_BO' # Additional options
                   )
trace5 = go.Scatter(
                    y=df3['AverageReturn'], # Data
                    mode='lines', name='training_perform_trained_in_real' # Additional options
                   )
trace6 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e3), # Data
                    mode='lines', name='testing_perform_trained_in_real' # Additional options
                   )
layout = go.Layout(title='Swimmer TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2], layout=layout)
# fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [9]:
with open(r"returns_Swimmer-v1_iter_200_2017-09-07 13:43:26.643265", "rb") as input_file:
    e = pickle.load(input_file)
with open(r"returns_Swimmer-v1_iter_200_2017-09-07 14:29:40.011605", "rb") as input_file:
    e2 = pickle.load(input_file)

datafile =  '../../data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0003/progress.csv'
df = pd.read_csv(datafile)

datafile2 =  '../../data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0006/progress.csv'
df2 = pd.read_csv(datafile2)

trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training' # Additional options
                   )
trace２ = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e), # Data
                    mode='lines', name='testing' # Additional options
                   )
trace3 = go.Scatter(
                    y=df2['AverageReturn'], # Data
                    mode='lines', name='training2' # Additional options
                   )
trace4 = go.Scatter(
                    x=np.array(range(10,1000,10)),
                    y=np.array(e2), # Data
                    mode='lines', name='testing2' # Additional options
                   )
layout = go.Layout(title='Swimmer TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [7]:
datafile =  '../../data/local/trpo-HalfCheetah-v1-32-32-4000-40/trpo_HalfCheetah-v1_32_32_4000_40_2017_09_05_20_31_58_0023/progress.csv'
df = pd.read_csv(datafile)
trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training' # Additional options
                   )
trace２ = go.Scatter(
                    y=np.array([10.0, 4.0, 5.0, 4.0, 3.0, 12.0, 30.0, 66.0, 15.0, 45.0, 78.0, 102.0, 29.0, 55.0, 81.0, 115.0, 133.0, 103.0, 166.0, 170.0, 154.0, 26.0, 486.0, 1000.0, 306.0, 283.0, 819.0, 394.0, 478.0, 524.0, 873.0, 893.0, 1000.0, 1000.0, 802.0, 293.0, 1000.0, 1000.0, 224.0, 604.0]), # Data
                    mode='lines', name='testing (mass*0.6)' # Additional options
                   )
layout = go.Layout(title='HalfCheetah TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [4]:
datafile =  '../../data/local/trpo-ip-32-32-4000-100/trpo_ip_32_32_4000_100_2017_09_05_17_48_37_0003/progress.csv'
df = pd.read_csv(datafile)
trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='training' # Additional options
                   )
trace２ = go.Scatter(
                    y = np.array([11.0, 4.0, 3.0, 5.0, 5.0, 7.0, 31.0, 35.0, 6.0, 50.0, 79.0, 92.0, 38.0, 62.0, 144.0, 139.0, 163.0, 136.0, 94.0, 15.0, 225.0, 228.0, 443.0, 283.0, 805.0, 68.0, 761.0, 1000.0, 52.0, 1000.0, 1000.0, 694.0, 1000.0, 1000.0, 744.0, 1000.0, 1000.0, 7.0, 72.0, 1000.0, 594.0, 1000.0, 214.0, 110.0, 1000.0, 565.0, 185.0, 557.0, 246.0, 1000.0, 1000.0, 280.0, 545.0, 1000.0, 1000.0, 1000.0, 451.0, 857.0, 255.0, 312.0, 462.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 325.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 462.0, 549.0, 1000.0, 233.0, 1000.0, 1000.0, 1000.0, 733.0]
),
                    mode='lines', name='testing (mass*0.6)' # Additional options
                   )
layout = go.Layout(title='Inverted Pendulum TRPO Perfomance',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1, trace2], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [3]:
datafile =  '../../data/local/trpo-ip-32-32-4000-100/trpo_ip_32_32_4000_100_2017_09_05_17_48_37_0001/progress.csv'
df = pd.read_csv(datafile)
trace1 = go.Scatter(
                    y=df['AverageReturn'], # Data
                    mode='lines', name='logx' # Additional options
                   )
layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)